In [1]:
import json

with open("./params.json", mode = "r", encoding = "utf-8") as f:
    data = json.load(f)
    seed_val = data["seed_val"]
    model_path = data["model_path"]
    dataset_path_train = data["dataset_path"]["train"]
    dataset_path_val = data["dataset_path"]["validation"]
    dataset_path_test = data["dataset_path"]["test"]
    num_single_sample_timesteps = data["num_single_sample_timesteps"]
    window_stride = data["window_stride"]
    input_window_length = data["input_window_length"]
    label_window_length = data["label_window_length"]
    input_features = data["input_features"]
    label_features = data["label_features"]
    relative_attention_num_buckets = data["relative_attention_num_buckets"]
    embedding_dim = data["embedding_dim"]
    num_attention_head = data["num_attention_head"]
    num_encoder_layers = data["num_encoder_layers"]
    num_decoder_layers = data["num_decoder_layers"]
    position_wise_nn_dim = data["position_wise_nn_dim"]
    dropout = data["dropout"]
    batch_size = data["batch_size"]
    epochs = data["epochs"]
    learning_rate = data["learning_rate"]

In [2]:
import random
import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(seed_val)
random.seed(seed_val)
np.random.seed(seed_val)

In [3]:
from torch.utils.data import DataLoader
from torchmetrics.regression import R2Score
from datetime import datetime

from utils.pipeline.Data import get_mean_std_respected_temporal, WindowedIterableDataset
from utils.pipeline.Model import TimeSeriesHuggingFaceTransformer
from utils.pipeline.Run import train, validate
from utils.pipeline.Monitor import Overfit

/users/labnet5/gr5/abahari/Documents/Thesis/src/thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
##### TRAIN #####
# stats = get_mean_std(
#     dataset_path = dataset_path_train,
#     cols = input_features
# )

input_stats = get_mean_std_respected_temporal(
    dataset_path = dataset_path_train,
    cols = input_features,
    num_single_sample_timesteps = num_single_sample_timesteps,
    input_window_len = input_window_length,
    label_window_len = label_window_length,
    window_stride = window_stride
)
if(set(input_features) == set(label_features)):
    output_stats = input_stats
else:
    output_stats = get_mean_std_respected_temporal(
        dataset_path = dataset_path_train,
        cols = label_features,
        num_single_sample_timesteps = num_single_sample_timesteps,
        input_window_len = input_window_length,
        label_window_len = label_window_length,
        window_stride = window_stride
    )

df_train = WindowedIterableDataset(
    dataset_path = dataset_path_train,
    input_stats = input_stats,
    label_stats = output_stats,
    input_features = input_features,
    label_features = label_features,
    num_single_sample_timesteps = num_single_sample_timesteps,
    stride = window_stride,
    input_window_length = input_window_length,
    label_window_length = label_window_length
)

data_loader_train = DataLoader(
    df_train,
    batch_size = batch_size,
    # num_workers = 0,
    # prefetch_factor = 12,
    # persistent_workers = False,
    pin_memory = True
)



##### VALIDATION #####
df_val = WindowedIterableDataset(
    dataset_path = dataset_path_val,
    input_stats = input_stats,
    label_stats = output_stats,
    input_features = input_features,
    label_features = label_features,
    num_single_sample_timesteps = num_single_sample_timesteps,
    stride = window_stride,
    input_window_length = input_window_length,
    label_window_length = label_window_length
)

data_loader_val = DataLoader(
    df_val,
    batch_size = batch_size,
    # num_workers = 0,
    # prefetch_factor = 12,
    # persistent_workers = False,
    pin_memory = True
)



##### TEST #####
df_test = WindowedIterableDataset(
    dataset_path = dataset_path_test,
    input_stats = input_stats,
    label_stats = output_stats,
    input_features = input_features,
    label_features = label_features,
    num_single_sample_timesteps = num_single_sample_timesteps,
    stride = window_stride,
    input_window_length = input_window_length,
    label_window_length = label_window_length
)

data_loader_test = DataLoader(
    df_test,
    batch_size = batch_size,
    # num_workers = 0,
    # prefetch_factor = 12,
    # persistent_workers = False,
    pin_memory = True
)

In [5]:
model = TimeSeriesHuggingFaceTransformer(
    input_window_len = input_window_length,
    output_window_len = label_window_length,
    input_dim = len(input_features),
    output_dim = len(label_features),
    d_model = embedding_dim,
    num_head = num_attention_head,
    num_encoder_layers = num_encoder_layers,
    num_decoder_layers = num_encoder_layers,
    position_wise_ffn_dim = position_wise_nn_dim,
    relative_attention_num_buckets = relative_attention_num_buckets,
    dropout = dropout
).to(device)

overfit_monitor = Overfit()

print(f"Number of trainable parameters in the model: {sum(p.numel() for p in model.parameters() if p.requires_grad)}\n")

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr = learning_rate
)

train_r2 = R2Score(multioutput = "uniform_average").to(device)
val_r2 = R2Score(multioutput = "uniform_average").to(device)

train_per_timestep_r2 = [R2Score(multioutput = "uniform_average").to(device) for _ in range(label_window_length)]
val_per_timestep_r2 = [R2Score(multioutput = "uniform_average").to(device) for _ in range(label_window_length)]

overfit_count = 0

for epoch in range(epochs):
    train_loss, train_r2_value = train(
        model = model,
        optimizer = optimizer,
        criterion = criterion,
        r2 = train_r2,
        per_timestep_r2 = train_per_timestep_r2,
        data_loader = data_loader_train,
        device = device,
        epoch = epoch,
        total_epochs = epochs
    )

    val_loss, val_r2_value = validate(
        model = model,
        criterion = criterion,
        r2 = val_r2,
        per_timestep_r2 = val_per_timestep_r2,
        data_loader = data_loader_val,
        device = device,
        epoch = epoch,
        total_epochs = epochs
    )

    # if(overfit_monitor.check(epoch = epoch, train_loss = train_loss, val_loss = val_loss)):
    #     break

Number of trainable parameters in the model: 1170949



Epoch 1/20: 717it [00:58, 12.35it/s, train_loss=0.030148]


Epoch [1/20], Train Loss: 0.099944, Train R2: 0.920496

Worst 5 Time-Steps Train R2:
    Time-step 1: R2 = 0.761971
    Time-step 2: R2 = 0.895655
    Time-step 3: R2 = 0.907683
    Time-step 4: R2 = 0.914820
    Time-step 50: R2 = 0.918148
Best 5 Time-Steps Train R2:
    Time-step 15: R2 = 0.928165
    Time-step 16: R2 = 0.928126
    Time-step 20: R2 = 0.928121
    Time-step 14: R2 = 0.928081
    Time-step 19: R2 = 0.927928



Epoch 1/20: 207it [00:32,  6.42it/s, val_loss=0.416815]


Epoch [1/20], Val Loss: 0.838959, Val R2: 0.387097

Worst 5 Time-Steps Val R2:
    Time-step 50: R2 = -0.298190
    Time-step 49: R2 = -0.276344
    Time-step 48: R2 = -0.249180
    Time-step 47: R2 = -0.220187
    Time-step 46: R2 = -0.192943
Best 5 Time-Steps Val R2:
    Time-step 1: R2 = 0.970712
    Time-step 2: R2 = 0.956640
    Time-step 3: R2 = 0.940655
    Time-step 4: R2 = 0.924485
    Time-step 5: R2 = 0.907421

-----------------------------------------------------------------



...

Epoch [19/20], Val Loss: 0.466958, Val R2: 0.645217

Worst 5 Time-Steps Val R2:
    Time-step 50: R2 = 0.266501
    Time-step 49: R2 = 0.280038
    Time-step 48: R2 = 0.292754
    Time-step 47: R2 = 0.306710
    Time-step 46: R2 = 0.321402
Best 5 Time-Steps Val R2:
    Time-step 1: R2 = 0.986019
    Time-step 2: R2 = 0.977625
    Time-step 3: R2 = 0.968279
    Time-step 4: R2 = 0.957999
    Time-step 5: R2 = 0.947578

-----------------------------------------------------------------



Epoch 20/20: 717it [00:56, 12.76it/s, train_loss=0.013840]


Epoch [20/20], Train Loss: 0.020814, Train R2: 0.983243

Worst 5 Time-Steps Train R2:
    Time-step 1: R2 = 0.972956
    Time-step 2: R2 = 0.980486
    Time-step 3: R2 = 0.982092
    Time-step 4: R2 = 0.982702
    Time-step 5: R2 = 0.982961
Best 5 Time-Steps Train R2:
    Time-step 33: R2 = 0.983737
    Time-step 34: R2 = 0.983724
    Time-step 23: R2 = 0.983703
    Time-step 31: R2 = 0.983701
    Time-step 20: R2 = 0.983700



Epoch 20/20: 207it [00:32,  6.47it/s, val_loss=0.312622]

Epoch [20/20], Val Loss: 0.416274, Val R2: 0.678230

Worst 5 Time-Steps Val R2:
    Time-step 50: R2 = 0.329880
    Time-step 49: R2 = 0.344385
    Time-step 48: R2 = 0.357542
    Time-step 47: R2 = 0.371504
    Time-step 46: R2 = 0.385730
Best 5 Time-Steps Val R2:
    Time-step 1: R2 = 0.988314
    Time-step 2: R2 = 0.979743
    Time-step 3: R2 = 0.971093
    Time-step 4: R2 = 0.961890
    Time-step 5: R2 = 0.952642

-----------------------------------------------------------------



In [6]:
torch.save(model, model_path)

description = '''
dataset: random
bos_projector: non-linear (1 LeakyReLU)
bos_input: encoder hidden state of last input time-step
positional encoding: sin, cos
Loss: MSE
seed_val: 20
num_single_sample_timesteps: 1000
input_window_len: 100
label_window_len: 50
window_stride: 10
relative_attention_num_buckets: 64
embedding_dim: 128
num_attention_head: 8
num_encoder_layers: 5
num_decoder_layers: 5
position_wise_nn_dim: 64
dropout: 0.5
batch_size: 128
epochs: 20
learning_rate: 0.0005
psi_e, b_e, psi_plus, b_plus, u_list, eta_list -> psi_e, b_e, psi_plus, b_plus, u_list

'''

with open("./results/experiments.txt", mode = "a") as f:
    f.write(str(datetime.now()).replace(" ", "-") + "\n")
    f.write(f"{model_path}\n\n")
    f.write(f"Training Loss: {train_loss}, Training R2: {train_r2_value}\n")
    f.write(f"Validation Loss: {val_loss}, Validation R2: {val_r2_value}\n")
    f.write(description)
    f.write("==========================================\n")